In [ ]:
import cv2
import numpy as np


cam = cv2.VideoCapture(0)
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)


MIN_MATCH_COUNT=30

detector = cv2.xfeatures2d.SIFT_create()
madfox = cv2.imread('madfox.jpg', cv2.IMREAD_GRAYSCALE)
kpmad, des1=detector.detectAndCompute(madfox,None) 

FLANN_INDEX_KDITREE=0
flannpar = dict(algorithm=FLANN_INDEX_KDITREE,tree=5)
flann = cv2.FlannBasedMatcher(flannpar, {})


cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

lower = 0
upper = 1

def auto_canny(image, sigma=0.33):
    # compute the median of the single channel pixel intensities
    v = np.median(image)

    # apply automatic Canny edge detection using the computed median
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edged = cv2.Canny(image, lower, upper)

while(True):
    # Capture frame-by-frame
    print("New frame")
    ret, frame = cam.read()

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # A gaussian blur to get rid of the noise in the image
    blur = cv2.GaussianBlur(gray,(5,5),0)
    #blur = gray
    # Detect the edges present in the image
    #bordas = auto_canny(blur)
   
    # Match do MadFox
    bordas = blur
    kpgray, des2=detector.detectAndCompute(blur,None)
    matches=flann.knnMatch(des1,des2,k=2)
    Pcorrespondentes = []
    for m, n in matches:
        if m.distance < 0.7*n.distance:
            Pcorrespondentes.append(m)

    if len(Pcorrespondentes)>MIN_MATCH_COUNT:
        
            src_pts = np.float32([ kpmad[m.queryIdx].pt for m in Pcorrespondentes ]).reshape(-1,1,2)
            dst_pts = np.float32([ kpgray[m.trainIdx].pt for m in Pcorrespondentes ]).reshape(-1,1,2)
            M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
            matchesMask = mask.ravel().tolist()
            
            
            h,w=madfox.shape
            pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)

            dst = cv2.perspectiveTransform(pts,M)
            img2b = cv2.polylines(blur,[np.int32(dst)],True,255,3, cv2.LINE_AA)
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(img2b,'Mad Fox',(300,50), font, 1.5,(255,255,255),2,cv2.LINE_AA)

            
            
    else:
        print("Not enough matches are found - %d/%d" % (len(Pcorrespondentes),MIN_MATCH_COUNT))
        matchesMask = None
        


    draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                   singlePointColor = None,
                   matchesMask = matchesMask, # draw only inliers
                   flags = 2)

#Detecta Circulo

    
    circles = []


    # Obtains a version of the edges image where we can draw in color
    bordas_color = cv2.cvtColor(bordas, cv2.COLOR_GRAY2BGR)

    # HoughCircles - detects circles using the Hough Method. For an explanation of
    # param1 and param2 please see an explanation here http://www.pyimagesearch.com/2014/07/21/detecting-circles-images-using-opencv-hough-circles/
    circles = None
    circles=cv2.HoughCircles(bordas,cv2.HOUGH_GRADIENT,2,40,param1=50,param2=100,minRadius=10,maxRadius=60)

    if circles is not None:
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(bordas_color,'Circulo',(100,50), font, 1.5,(255,255,255),2,cv2.LINE_AA)
        circles = np.uint16(np.around(circles))

        for i in circles[0,:]:
            print(i)
            # draw the outer circle
            # cv2.circle(img, center, radius, color[, thickness[, lineType[, shift]]])
            cv2.circle(bordas_color,(i[0],i[1]),i[2],(0,255,0),2)
            # draw the center of the circle
            cv2.circle(bordas_color,(i[0],i[1]),2,(0,0,255),3)




    # Display the resulting frame
    cv2.imshow('Detector de circulos',bordas_color)
 
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

New frame
[177 155  52]
New frame
[183 157  48]
New frame
[169 179  50]
[389 255  42]
[331 209  39]
New frame
[167 189  49]
[315 217  39]
New frame
[161 195  50]
[383 175  59]
[313 219  40]
New frame
[167 189  52]
[387 173  55]
New frame
[169 185  53]
[393 271  54]
[325 231  58]
New frame
[163 187  49]
[381 171  59]
New frame
[159 195  47]
New frame
[159 191  46]
[303 213  38]
New frame
[157 193  48]
[367 177  55]
New frame
[159 189  46]
[369 177  59]
[359 261  36]
New frame
[165 195  45]
[371 181  55]
New frame
[131 217  57]
New frame
[111 275  58]
New frame
[ 87 277  55]
New frame
[ 91 281  50]
[311 271  57]
New frame
[ 83 279  53]
[309 273  58]
New frame
[117 261  55]
[331 251  55]
[269 305  56]
New frame
[149 235  44]
[357 229  55]
[389 275  47]
New frame
[151 255  49]
[359 237  52]
New frame
[143 273  44]
[347 257  57]
New frame
[141 281  41]
[335 267  55]
New frame
[143 287  44]
[335 277  52]
[401 253  48]
[355 327  42]
[283 325  42]
New frame
[145 287  43]
[339 279  53]
[395 263